In [0]:
from pyspark.sql import functions as F
    
# 1. Ler os dados da camada Bronze
df = spark.read.table("stg_sus_establishments")

todas_colunas = df.columns

# 2. Varredura para contar registros válidos (não nulos e não vazios)
contagem_dict = df.select([
    F.count(
        F.when(
            (F.col(c).isNotNull()) & (F.trim(F.col(c)) != ""), 
            c
        )
    ).alias(c)
    for c in todas_colunas
]).collect()[0].asDict()

# 3. Identificar colunas vazias
colunas_completamente_nulas = [col for col, count in contagem_dict.items() if count == 0]

# 4. Seleciona apenas colunas fora da lista de colunas vazias
colunas_para_manter = [c for c in todas_colunas if c not in colunas_completamente_nulas]

print({"colunas_completamente_nulas": colunas_completamente_nulas, "colunas_para_manter": colunas_para_manter})
